In [25]:
import geopandas as gpd
import pandas as pd
from shapely import Point, LineString, MultiLineString
from shapely import distance, from_wkt, intersects, buffer, set_precision, shortest_line
from shapely.ops import split, snap
from shapely.ops import nearest_points
from tqdm import tqdm
import ast

In [26]:
# use settings for Lima or London here


# Lima ~ 15 min calculation

crs = 32718

roads_data_path = "data/roads_lima.geojson"
bus_route_path = 'data/processed_results_lima.csv'

output_path = "lima_lanes.csv"


# # London

# crs = 32630

# roads_data_path = "data/roads_london.geojson"
# bus_route_path = 'data/processed_results_london.csv'

# output_path = "london_lanes.csv"

In [27]:
df = pd.read_csv(bus_route_path, delimiter=',', header=None, names=["start_id", "stop_id", "geometry"], skiprows=1, index_col=False)
osm_gdf = gpd.read_file(roads_data_path)

In [28]:
data = {
    "start_ids": df["start_id"],
    "end_ids": df["stop_id"]
}
geometry = [from_wkt(geom) for geom in df["geometry"]]
out_gdf = gpd.GeoDataFrame(data, geometry=geometry, crs=crs)
out_gdf = out_gdf.to_crs(crs)

In [29]:
osm_gdf = osm_gdf.to_crs(crs)
osm_gdf["tags"] = osm_gdf["tags"].apply(lambda x: ast.literal_eval(x))
osm_gdf['lanes'] = osm_gdf['tags'].apply(
    lambda x: x.get('lanes') if isinstance(x, dict) and 'lanes' in x else None
)
osm_gdf['oneway'] = osm_gdf['tags'].apply(
    lambda x: x.get('oneway') if isinstance(x, dict) and 'oneway' in x else None
)

In [30]:
osm_sindex = osm_gdf.sindex

length_over_3_lanes = []
length_3_or_less_lanes = []
none_length = 0

clipped_rows = []

for index, row in tqdm(out_gdf.iterrows(), total=out_gdf.shape[0], desc="Processing rows", unit="row"):
    original_length = row['geometry'].length
    buffered_geom = row['geometry'].buffer(distance=6)
    possible_matches_index = list(osm_sindex.intersection(buffered_geom.bounds))
    possible_matches = osm_gdf.iloc[possible_matches_index]
    intersecting = possible_matches[possible_matches.intersects(buffered_geom)]

    length_more_3 = 0
    len_less_3 = 0
    
    if not intersecting.empty:
        clipped = intersecting.copy()
        
        # Buffer and intersection calculations
        within_buffer = clipped[clipped['geometry'].within(buffered_geom)]
        clipped['geometry'] = clipped['geometry'].intersection(buffered_geom)
        clipped = clipped[clipped.length >= 10]
        
        # Aggregate lengths based on lanes
        for _, clipped_row in clipped.iterrows():
            lanes = clipped_row['lanes']
            oneway = clipped_row['oneway']
            length = clipped_row['geometry'].length

            if lanes is None:
                none_length += length
            
            if (lanes is not None and lanes.isdigit() and int(lanes) > 3) or (lanes is not None and lanes.isdigit() and int(lanes) > 1 and oneway == "yes"):
                length_more_3 += length
            else:
                len_less_3 += length
            
    length_over_3_lanes.append(length_more_3)
    length_3_or_less_lanes.append(len_less_3)


Processing rows: 100%|██████████████████| 63230/63230 [07:50<00:00, 134.28row/s]


In [31]:
out_gdf["original_length"] = out_gdf["geometry"].length
out_gdf["len_two_lanes"] = length_3_or_less_lanes
out_gdf["len_more_than_two_lanes"] = length_over_3_lanes

In [32]:
def adjust_lengths(row):
    L1, L2, true_length = row['len_two_lanes'], row['len_more_than_two_lanes'], row['original_length']
    total_length = L1 + L2

    if total_length == 0:
        return pd.Series([0, 0])
    
    scaling_factor = true_length / total_length
    L1_adjusted = L1 * scaling_factor
    L2_adjusted = L2 * scaling_factor
    return pd.Series([L1_adjusted, L2_adjusted])

In [33]:
out_gdf = out_gdf.drop(columns=["geometry", "original_length"])
out_gdf.to_csv(output_path, index=False)